In [1]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error

In [2]:
#read file
df_data = pd.read_csv("data/dataset_mood_smartphone.csv")
df_data.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [3]:
prev_day="2014-02-26"
new_data=pd.DataFrame(columns=['id', 'time', 'var',"value"])
temp_list=[]
for index, row in df_data.iterrows():
    date=row["time"].split(" ")[0]
    if date != prev_day:
        new_data.loc[index]=[df_data.loc[index-1]["id"], prev_day , df_data.loc[index-1]["variable"], statistics.mean(temp_list)]
        temp_list=[row["value"]]
    else:
        temp_list.append(row["value"])
    prev_day=date

In [ ]:
ids = list(np.unique(np.array(df_data["id"])))

In [ ]:
for id in ids:
    df_mood_id = new_data.loc[(new_data['var'] == "mood") & (new_data['id']==id)]
    df_time_value = df_mood_id[['id','time',"value"]].reset_index()
    df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

    X = []

    prev_index = 0
    for i, row in df_time_value.iterrows():
        delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
        if delta < -1:
            for n in range(abs(delta)-1):
                X.append(None)
            X.append(df_time_value['value'].iloc[i])
        elif delta > 0:
            for n in range(365-delta-1):
                X.append(None)
            X.append(df_time_value['value'].iloc[i])
        else:
            X.append(df_time_value['value'].iloc[i])
        prev_index = i

    #Set up the input for ARIMA
    # index = df_time_value[["time"]].values
    # data_np = df_time_value["value"].values
    series = pd.Series(X)
    X= series.values
    print(X)
#     np.savetxt("dataset_arima.txt", X)
    X = [x for x in X if str(x) != 'nan']
    
    size = int(len(X) * 0.66)
    train_val, test = X[0:size], X[size:len(X)]
    
    size_train = int(len(train_val) * 0.75)
    train, val = train_val[0:size_train], train_val[size_train:len(train_val)]
    

#   hyperparameter selection
    history = [x for x in train]
    range_p = [0,1,2,3]
    range_d = [0,1,2,3]
    range_q = [0,1,2,3]
    hyperparameter_error = list()
    for p in range_p:
        for d in range_d:
            for q in range_q:
                try:
                    predictions = list()
                    for t in range(len(val)):
                        model = ARIMA(history, order=(p,d,q))
                        model_fit = model.fit(disp=-1, start_ar_lags=13)
                        output = model_fit.forecast()
                        yhat = output[0]
                        predictions.append(yhat)
                        obs = val[t]
                        history.append(obs)
                    error = mean_squared_error(val, predictions)
                    hyperparameter_error.append([error,p,d,q])
                    print(p,d,q, error)
                except:
                    pass
    best_hyperparameter = [1,0,0,0]
    for i in hyperparameter_error:
        if i[0] < best_hyperparameter[0]:
            best_hyperparameter = i
    
    print('Error: \t'+ str(best_hyperparameter[0]))
    print('Best hyperparameter (p,d,q):\t',best_hyperparameter[1:4])

    
    history = [x for x in train_val]
    
    predictions = list()
    for t in range(len(test)):
        #order=(p,d,q) 
        model = ARIMA(history, order=tuple(best_hyperparameter[1:4]))#tunne parameters!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        model_fit = model.fit(disp=-1)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
#         print('predicted=%f, expected=%f' % (yhat, obs))
    error = mean_squared_error(test, predictions)
    plt.plot(test, predictions)
    plt.show()
    print('Test MSE: %.3f' % error)


[6.25       6.33333333        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan 6.2
 6.4        6.8        6.         6.75       6.6        7.
 6.4        8.         7.5        7.4        6.         6.5
 6.4        6.2        6.8        6.5        6.5        6.6
 7.4        7.4        7.         7.25       8.         7.2
 6.4        7.2        6.6        6.8        7.8        7.25
 7.6        7.4        7.6        7.6        7.2        7.6
 7.5        7.6        7.         7.8        8.         7.6
 8.         8.        ]
0 0 0 0.31988079833256283
0 0 1 0.2527313565951403
0 0 2 0.25368480600538257
0 0 3 0.24209493702399354
0 1 0 0.3796747954572808
0 1 1 0.24803226922067867
0 1 2 0.23186751587123222
0 1 3 0.2300216999877671
0 2 0 0.9877691298268199


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0 2 1 0.386554829462162
1 0 0 0.21684298712267552
1 0 1 0.19946226551549817


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


1 0 2 0.21178222020691867


...

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:676: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:650: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:651: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:654: RuntimeWarning: invalid value encountered in log
  R_mat, T_mat)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:

2 1 0 0.29541059169868167
2 1 1 0.2079114668477888
2 1 2 0.1957277588588045
2 2 0 0.4591838035349042
2 2 1 0.29884851753074415
2 2 2 0.281693558320557
3 0 0 0.20006530358673413
3 0 1 0.2000072222842649


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:650: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:650: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:651: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:651: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:606: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:607: RuntimeWarning: overfl

3 0 2 0.18734529505301314


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [ ]:
df_mood_id = new_data.loc[(new_data['var'] == "mood")]
df_time_value = df_mood_id[['id','time',"value"]].reset_index()
df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

X = []

id = 'AS14.01'
prev_index = 0
for i, row in df_time_value.iterrows():
    delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
    if delta < -1:
        for n in range(abs(delta)-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    elif delta > 0:
        for n in range(365-delta-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    else:
        X.append(df_time_value['value'].iloc[i])
    prev_index = i

#Set up the input for ARIMA
# index = df_time_value[["time"]].values
# data_np = df_time_value["value"].values
series = pd.Series(X)
X= series.values
X = [x for x in X if str(x) != 'nan']
# X = [x for x in X]
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()
for t in range(len(test)):
    #order=(p,d,q) 
    model = ARIMA(history, order=(1,1,1))#tunne parameters!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

In [ ]:
df_time_value.shape